**March Machine Learning Mania 2025**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss

# Load Women's Data with absolute paths from USB
teams_womens = pd.read_csv(r'D:\march-machine-learning-mania-2025\WTeams.csv')
reg_season_womens = pd.read_csv(r'D:\march-machine-learning-mania-2025\WRegularSeasonDetailedResults.csv')
tournament_results_womens = pd.read_csv(r'D:\march-machine-learning-mania-2025\WNCAATourneyDetailedResults.csv')
tournament_seeds_womens = pd.read_csv(r'D:\march-machine-learning-mania-2025\WNCAATourneySeeds.csv')

# Load Men's Data with absolute paths from USB
teams_mens = pd.read_csv(r'D:\march-machine-learning-mania-2025\MTeams.csv')
reg_season_mens = pd.read_csv(r'D:\march-machine-learning-mania-2025\MRegularSeasonDetailedResults.csv')
tournament_results_mens = pd.read_csv(r'D:\march-machine-learning-mania-2025\MNCAATourneyDetailedResults.csv')
tournament_seeds_mens = pd.read_csv(r'D:\march-machine-learning-mania-2025\MNCAATourneySeeds.csv')

# Load Sample Submission with absolute path from USB
sample_submission = pd.read_csv(r'D:\march-machine-learning-mania-2025\SampleSubmissionStage2.csv')
print(f"Sample Submission Shape: {sample_submission.shape}")

# Split into Men's and Women's Matchups
sample_submission[['Season', 'Team1', 'Team2']] = sample_submission['ID'].str.split('_', expand=True).astype(int)
womens_submission = sample_submission[(sample_submission['Team1'].between(3101, 3613)) & (sample_submission['Team2'].between(3101, 3613))]
mens_submission = sample_submission[(sample_submission['Team1'].between(1101, 1484)) & (sample_submission['Team2'].between(1101, 1484))]
print(f"Women's Submission Shape: {womens_submission.shape}")
print(f"Men's Submission Shape: {mens_submission.shape}")


Beginning with the women's data, we group by season and winning team ID and losing team ID. This allows for the calculation of a new column to calculate the win percentage for each team ID.

In [ ]:
# Compute Team Statistics for Women
wins_w = reg_season_womens.groupby(['Season', 'WTeamID']).size().reset_index(name='Wins').rename(columns={'WTeamID': 'TeamID'})
losses_w = reg_season_womens.groupby(['Season', 'LTeamID']).size().reset_index(name='Losses').rename(columns={'LTeamID': 'TeamID'})
team_stats_w = pd.merge(wins_w, losses_w, on=['Season', 'TeamID'], how='outer').fillna(0)
team_stats_w['WinPercent'] = team_stats_w['Wins'] / (team_stats_w['Wins'] + team_stats_w['Losses'])

Margin of Victory (MOV) Calculation. A new column titled MOV is created from the computation of 'PointsScored' and 'PointsAllowed.' 

In [ ]:
points_scored_w = reg_season_womens.groupby(['Season', 'WTeamID'])['WScore'].sum().reset_index(name='PointsScored').rename(columns={'WTeamID': 'TeamID'})
points_allowed_w = reg_season_womens.groupby(['Season', 'WTeamID'])['LScore'].sum().reset_index(name='PointsAllowed').rename(columns={'WTeamID': 'TeamID'})
team_stats_w = pd.merge(team_stats_w, points_scored_w, on=['Season', 'TeamID'], how='outer').fillna(0)
team_stats_w = pd.merge(team_stats_w, points_allowed_w, on=['Season', 'TeamID'], how='outer').fillna(0)
team_stats_w['MOV'] = (team_stats_w['PointsScored'] - team_stats_w['PointsAllowed']) / (team_stats_w['Wins'] + team_stats_w['Losses'])

Identify seed numbers for each team each season. This is to eventually to give weights to team outcomes. Ex: (#13 seed beating #2 > #2 seed beating #13 seed)

In [ ]:
team_stats_w = team_stats_w.merge(tournament_seeds_womens[['Season', 'TeamID', 'Seed']], on=['Season', 'TeamID'], how='left')
team_stats_w['SeedNum'] = team_stats_w['Seed'].str.extract(r'(\d+)').astype(float).fillna(16)


Offensive and Defensive Strength using Points Scored/Allowed and Wins/Losses.

In [ ]:
team_stats_w['OffenseStrength'] = team_stats_w['PointsScored'] / (team_stats_w['Wins'] + team_stats_w['Losses']).replace(0, 1)
team_stats_w['DefenseStrength'] = team_stats_w['PointsAllowed'] / (team_stats_w['Wins'] + team_stats_w['Losses']).replace(0, 1)

The win percentage of the last 5 games can be an indicator on how well a team is performing heading into a tournament. This cell sorts the values of Season, TeamID and DayNum then groups by Season and TeamID using the .tail() function to identify the last 5 rows (games). The newly assigned column "Result" is averaged for the win rate of the last 5 games. 

In [ ]:
last_five_games_w = pd.concat([
    reg_season_womens[['Season', 'DayNum', 'WTeamID', 'LTeamID']].assign(TeamID=lambda x: x['WTeamID'], Result=1),
    reg_season_womens[['Season', 'DayNum', 'WTeamID', 'LTeamID']].assign(TeamID=lambda x: x['LTeamID'], Result=0)
])
last_five_games_w = last_five_games_w.sort_values(['Season', 'TeamID', 'DayNum']).groupby(['Season', 'TeamID']).tail(5)
last_five_winrate_w = last_five_games_w.groupby(['Season', 'TeamID'])['Result'].mean().reset_index(name='LastFiveWinRate')
team_stats_w = team_stats_w.merge(last_five_winrate_w, on=['Season', 'TeamID'], how='left').fillna({'LastFiveWinRate': 0.5})

Turnover Ratios are an important statistic for a machine learning algorithm to learn form. The common theme is to create a new data frame (named after what we are attempting to extract). Next we select which columns we are pulling data from (in this case we want the season, the winning team ID and the amount of turnovers the winning team ID had). The same is done for the losing team turnovers. Concatenating puts the winning and losing data frames on top of each other. This creates a new data frame of all the turnovers for winning and losing teams. .mean() averages them and then this new dat ais merged into team_stats. The team_stats data frame will be important for our algorithm later. 

In [ ]:
wins_turnovers_w = reg_season_womens[['Season', 'WTeamID', 'WTO']].rename(columns={'WTeamID': 'TeamID', 'WTO': 'Turnovers'})
losses_turnovers_w = reg_season_womens[['Season', 'LTeamID', 'LTO']].rename(columns={'LTeamID': 'TeamID', 'LTO': 'Turnovers'})
turnovers_data_w = pd.concat([wins_turnovers_w, losses_turnovers_w], ignore_index=True)
turnover_ratio_w = turnovers_data_w.groupby(['Season', 'TeamID'])['Turnovers'].mean().reset_index(name='TurnoverRatio')
team_stats_w = pd.merge(team_stats_w, turnover_ratio_w, on=['Season', 'TeamID'], how='left').fillna(0)

Rebound Margin is another influential statistic. This begins at the base level of finding how many rebounds winning and losing teams had. Concatenate these two new data frames then create a new column called 'TotalRebounds.' This is simply the sum of the offensive and defensive rebounds for the winning team. It was important to rb_margin_w.groupby after adding the rebound totals because this will seperate the total rebounds for each individual team from each prior season. Otherwise this would have simply added every rebound from every team as a singular value which would have been useless. The same thing is done foir the losing team rebounds 

In [ ]:
win_rebounds_w = reg_season_womens[['Season', 'WTeamID', 'WOR', 'WDR']].rename(columns={'WTeamID': 'TeamID', 'WOR': 'OffRebounds', 'WDR': 'DefRebounds'})
loss_rebounds_w = reg_season_womens[['Season', 'LTeamID', 'LOR', 'LDR']].rename(columns={'LTeamID': 'TeamID', 'LOR': 'OffRebounds', 'LDR': 'DefRebounds'})

#Winning team Total Reb
rb_margin_w = pd.concat([win_rebounds_w, loss_rebounds_w], ignore_index=True)
rb_margin_w['TotalRebounds'] = rb_margin_w['OffRebounds'] + rb_margin_w['DefRebounds']
team_rebounds_w = rb_margin_w.groupby(['Season', 'TeamID'])['TotalRebounds'].sum().reset_index(name='TeamTotalRebounds')
win_opp_rebound_w = reg_season_womens[['Season', 'WTeamID', 'LOR', 'LDR']].rename(columns={'WTeamID': 'TeamID'})
win_opp_rebound_w['OpponentRebounds'] = win_opp_rebound_w['LOR'] + win_opp_rebound_w['LDR']

#Losing Team Total Reb
loss_opp_rebound_w = reg_season_womens[['Season', 'LTeamID', 'WOR', 'WDR']].rename(columns={'LTeamID': 'TeamID'})
loss_opp_rebound_w['OpponentRebounds'] = loss_opp_rebound_w['WOR'] + loss_opp_rebound_w['WDR']
opponent_rebounds_w = pd.concat([win_opp_rebound_w, loss_opp_rebound_w], ignore_index=True)
opponent_rebounds_sum_w = opponent_rebounds_w.groupby(['Season', 'TeamID'])['OpponentRebounds'].sum().reset_index(name='OpponentTotalRebound')

#Merge to team_stats
rb_margin_teams_w = pd.merge(team_rebounds_w, opponent_rebounds_sum_w, on=['Season', 'TeamID'], how='outer')
rb_margin_teams_w = rb_margin_teams_w.merge(team_stats_w[['Season', 'TeamID', 'Wins', 'Losses']], on=['Season', 'TeamID'], how='left')
rb_margin_teams_w['GamesPlayed'] = rb_margin_teams_w['Wins'] + rb_margin_teams_w['Losses']
rb_margin_teams_w['TotalReboundMargin'] = rb_margin_teams_w['TeamTotalRebounds'] - rb_margin_teams_w['OpponentTotalRebound']
rb_margin_teams_w['ReboundMargin'] = rb_margin_teams_w['TotalReboundMargin'] / rb_margin_teams_w['GamesPlayed'].replace(0, 1)
team_stats_w = team_stats_w.merge(rb_margin_teams_w[['Season', 'TeamID', 'ReboundMargin']], on=['Season', 'TeamID'], how='left').fillna(0)

The same preprocessing methodology is followed for the men's data:

In [ ]:
wins_m = reg_season_mens.groupby(['Season', 'WTeamID']).size().reset_index(name='Wins').rename(columns={'WTeamID': 'TeamID'})
losses_m = reg_season_mens.groupby(['Season', 'LTeamID']).size().reset_index(name='Losses').rename(columns={'LTeamID': 'TeamID'})
team_stats_m = pd.merge(wins_m, losses_m, on=['Season', 'TeamID'], how='outer').fillna(0)
team_stats_m['WinPercent'] = team_stats_m['Wins'] / (team_stats_m['Wins'] + team_stats_m['Losses'])

points_scored_m = reg_season_mens.groupby(['Season', 'WTeamID'])['WScore'].sum().reset_index(name='PointsScored').rename(columns={'WTeamID': 'TeamID'})
points_allowed_m = reg_season_mens.groupby(['Season', 'WTeamID'])['LScore'].sum().reset_index(name='PointsAllowed').rename(columns={'WTeamID': 'TeamID'})
team_stats_m = pd.merge(team_stats_m, points_scored_m, on=['Season', 'TeamID'], how='outer').fillna(0)
team_stats_m = pd.merge(team_stats_m, points_allowed_m, on=['Season', 'TeamID'], how='outer').fillna(0)
team_stats_m['MOV'] = (team_stats_m['PointsScored'] - team_stats_m['PointsAllowed']) / (team_stats_m['Wins'] + team_stats_m['Losses'])

team_stats_m = team_stats_m.merge(tournament_seeds_mens[['Season', 'TeamID', 'Seed']], on=['Season', 'TeamID'], how='left')
team_stats_m['SeedNum'] = team_stats_m['Seed'].str.extract(r'(\d+)').astype(float).fillna(16)

team_stats_m['OffenseStrength'] = team_stats_m['PointsScored'] / (team_stats_m['Wins'] + team_stats_m['Losses']).replace(0, 1)
team_stats_m['DefenseStrength'] = team_stats_m['PointsAllowed'] / (team_stats_m['Wins'] + team_stats_m['Losses']).replace(0, 1)

last_five_games_m = pd.concat([
    reg_season_mens[['Season', 'DayNum', 'WTeamID', 'LTeamID']].assign(TeamID=lambda x: x['WTeamID'], Result=1),
    reg_season_mens[['Season', 'DayNum', 'WTeamID', 'LTeamID']].assign(TeamID=lambda x: x['LTeamID'], Result=0)
])
last_five_games_m = last_five_games_m.sort_values(['Season', 'TeamID', 'DayNum']).groupby(['Season', 'TeamID']).tail(5)
last_five_winrate_m = last_five_games_m.groupby(['Season', 'TeamID'])['Result'].mean().reset_index(name='LastFiveWinRate')
team_stats_m = team_stats_m.merge(last_five_winrate_m, on=['Season', 'TeamID'], how='left').fillna({'LastFiveWinRate': 0.5})

wins_turnovers_m = reg_season_mens[['Season', 'WTeamID', 'WTO']].rename(columns={'WTeamID': 'TeamID', 'WTO': 'Turnovers'})
losses_turnovers_m = reg_season_mens[['Season', 'LTeamID', 'LTO']].rename(columns={'LTeamID': 'TeamID', 'LTO': 'Turnovers'})
turnovers_data_m = pd.concat([wins_turnovers_m, losses_turnovers_m], ignore_index=True)
turnover_ratio_m = turnovers_data_m.groupby(['Season', 'TeamID'])['Turnovers'].mean().reset_index(name='TurnoverRatio')
team_stats_m = pd.merge(team_stats_m, turnover_ratio_m, on=['Season', 'TeamID'], how='left').fillna(0)

win_rebounds_m = reg_season_mens[['Season', 'WTeamID', 'WOR', 'WDR']].rename(columns={'WTeamID': 'TeamID', 'WOR': 'OffRebounds', 'WDR': 'DefRebounds'})
loss_rebounds_m = reg_season_mens[['Season', 'LTeamID', 'LOR', 'LDR']].rename(columns={'LTeamID': 'TeamID', 'LOR': 'OffRebounds', 'LDR': 'DefRebounds'})
rb_margin_m = pd.concat([win_rebounds_m, loss_rebounds_m], ignore_index=True)
rb_margin_m['TotalRebounds'] = rb_margin_m['OffRebounds'] + rb_margin_m['DefRebounds']
team_rebounds_m = rb_margin_m.groupby(['Season', 'TeamID'])['TotalRebounds'].sum().reset_index(name='TeamTotalRebounds')
win_opp_rebound_m = reg_season_mens[['Season', 'WTeamID', 'LOR', 'LDR']].rename(columns={'WTeamID': 'TeamID'})
win_opp_rebound_m['OpponentRebounds'] = win_opp_rebound_m['LOR'] + win_opp_rebound_m['LDR']
loss_opp_rebound_m = reg_season_mens[['Season', 'LTeamID', 'WOR', 'WDR']].rename(columns={'LTeamID': 'TeamID'})
loss_opp_rebound_m['OpponentRebounds'] = loss_opp_rebound_m['WOR'] + loss_opp_rebound_m['WDR']
opponent_rebounds_m = pd.concat([win_opp_rebound_m, loss_opp_rebound_m], ignore_index=True)
opponent_rebounds_sum_m = opponent_rebounds_m.groupby(['Season', 'TeamID'])['OpponentRebounds'].sum().reset_index(name='OpponentTotalRebound')
rb_margin_teams_m = pd.merge(team_rebounds_m, opponent_rebounds_sum_m, on=['Season', 'TeamID'], how='outer')
rb_margin_teams_m = rb_margin_teams_m.merge(team_stats_m[['Season', 'TeamID', 'Wins', 'Losses']], on=['Season', 'TeamID'], how='left')
rb_margin_teams_m['GamesPlayed'] = rb_margin_teams_m['Wins'] + rb_margin_teams_m['Losses']
rb_margin_teams_m['TotalReboundMargin'] = rb_margin_teams_m['TeamTotalRebounds'] - rb_margin_teams_m['OpponentTotalRebound']
rb_margin_teams_m['ReboundMargin'] = rb_margin_teams_m['TotalReboundMargin'] / rb_margin_teams_m['GamesPlayed'].replace(0, 1)
team_stats_m = team_stats_m.merge(rb_margin_teams_m[['Season', 'TeamID', 'ReboundMargin']], on=['Season', 'TeamID'], how='left').fillna(0)

Now it is time to prepare the women's data in a format that is suitable for machine learning. This takes all of the statistic calculated prior and assigns them to each team. That way the machine learning model will be able to see such stats in a standardized format for supervised learning. 

In [ ]:
# Prepare Training Data for Women
tourney_data_womens = tournament_results_womens.copy()
tourney_data_womens['Team1'] = np.where(tourney_data_womens['WTeamID'] < tourney_data_womens['LTeamID'], 
                                        tourney_data_womens['WTeamID'], tourney_data_womens['LTeamID'])
tourney_data_womens['Team2'] = np.where(tourney_data_womens['WTeamID'] < tourney_data_womens['LTeamID'], 
                                        tourney_data_womens['LTeamID'], tourney_data_womens['WTeamID'])
tourney_data_womens['Result'] = np.where(tourney_data_womens['WTeamID'] < tourney_data_womens['LTeamID'], 1, 0)

for feature in ["WinPercent", "MOV", "SeedNum", "OffenseStrength", "DefenseStrength", "LastFiveWinRate", 'TurnoverRatio', 'ReboundMargin']:
    tourney_data_womens[f"{feature}_Team1"] = tourney_data_womens.merge(team_stats_w, 
                                                                        left_on=['Season', 'Team1'], 
                                                                        right_on=['Season', 'TeamID'],
                                                                        how='left')[feature]
    tourney_data_womens[f"{feature}_Team2"] = tourney_data_womens.merge(team_stats_w, 
                                                                        left_on=['Season', 'Team2'], 
                                                                        right_on=['Season', 'TeamID'],
                                                                        how='left')[feature]

tourney_data_womens['SeedDiff'] = tourney_data_womens['SeedNum_Team1'] - tourney_data_womens['SeedNum_Team2']

X_w = tourney_data_womens[[ 
    'WinPercent_Team1', 'WinPercent_Team2', 'MOV_Team1', 'MOV_Team2',
    'SeedNum_Team1', 'SeedNum_Team2', 'OffenseStrength_Team1', 'OffenseStrength_Team2',
    'DefenseStrength_Team1', 'DefenseStrength_Team2', 'LastFiveWinRate_Team1', 'LastFiveWinRate_Team2',
    'SeedDiff', 'TurnoverRatio_Team1', 'TurnoverRatio_Team2', 'ReboundMargin_Team1', 'ReboundMargin_Team2'
]].fillna(0)
y_w = tourney_data_womens['Result']

The exact same thing is done for the men's data:

In [ ]:
tourney_data_mens = tournament_results_mens.copy()
tourney_data_mens['Team1'] = np.where(tourney_data_mens['WTeamID'] < tourney_data_mens['LTeamID'], 
                                      tourney_data_mens['WTeamID'], tourney_data_mens['LTeamID'])
tourney_data_mens['Team2'] = np.where(tourney_data_mens['WTeamID'] < tourney_data_mens['LTeamID'], 
                                      tourney_data_mens['LTeamID'], tourney_data_mens['WTeamID'])
tourney_data_mens['Result'] = np.where(tourney_data_mens['WTeamID'] < tourney_data_mens['LTeamID'], 1, 0)

for feature in ["WinPercent", "MOV", "SeedNum", "OffenseStrength", "DefenseStrength", "LastFiveWinRate", 'TurnoverRatio', 'ReboundMargin']:
    tourney_data_mens[f"{feature}_Team1"] = tourney_data_mens.merge(team_stats_m, 
                                                                    left_on=['Season', 'Team1'], 
                                                                    right_on=['Season', 'TeamID'],
                                                                    how='left')[feature]
    tourney_data_mens[f"{feature}_Team2"] = tourney_data_mens.merge(team_stats_m, 
                                                                    left_on=['Season', 'Team2'], 
                                                                    right_on=['Season', 'TeamID'],
                                                                    how='left')[feature]

tourney_data_mens['SeedDiff'] = tourney_data_mens['SeedNum_Team1'] - tourney_data_mens['SeedNum_Team2']

X_m = tourney_data_mens[[ 
    'WinPercent_Team1', 'WinPercent_Team2', 'MOV_Team1', 'MOV_Team2',
    'SeedNum_Team1', 'SeedNum_Team2', 'OffenseStrength_Team1', 'OffenseStrength_Team2',
    'DefenseStrength_Team1', 'DefenseStrength_Team2', 'LastFiveWinRate_Team1', 'LastFiveWinRate_Team2',
    'SeedDiff', 'TurnoverRatio_Team1', 'TurnoverRatio_Team2', 'ReboundMargin_Team1', 'ReboundMargin_Team2'
]].fillna(0)
y_m = tourney_data_mens['Result']

Now the easiest part is setting up the Random Forest and Extra Tress algorithm. Splitting the data is important so the model knows what to learning from and what to test it's knowledge on. Scaling the data is important so it can have context in value changes. Since some of out preprocessed data involves ratios in combination of whole number additives, this is important. Finally the Extra Trees and Random Forest are put together with hyper paramter tuning. After a lot of trial and error and RandomGridSearch, these were the most optimal hyperparameters for the data. After that, the model is fit to the data and logistical loss (how confident the model is in it's answers) are printed.

In [ ]:
# Split and Scale for Women
X_train_w, X_test_w, y_train_w, y_test_w = train_test_split(X_w, y_w, test_size=0.2, random_state=42)
scaler_w = StandardScaler()
X_train_w_scaled = scaler_w.fit_transform(X_train_w)
X_test_w_scaled = scaler_w.transform(X_test_w)

et_w = ExtraTreesClassifier(
    n_estimators=500,
    max_depth=8,
    min_samples_split=20,
    min_samples_leaf=10,
    max_features='sqrt',
    bootstrap=True,
    random_state=42,
    n_jobs=-1,
    criterion='entropy',
    class_weight={0: 1, 1: 1.2}
)

rf_w = RandomForestClassifier(
    n_estimators=500,
    max_depth=8,
    min_samples_split=20,
    min_samples_leaf=10,
    max_features='sqrt',
    bootstrap=True,
    random_state=42,
    n_jobs=-1,
    criterion='entropy',
    class_weight={0: 1, 1: 1.2}
)


# Fit models for Women
et_w.fit(X_train_w_scaled, y_train_w)
rf_w.fit(X_train_w_scaled, y_train_w)

et_pred_w = et_w.predict_proba(X_test_w_scaled)[:, 1]
rf_pred_w = rf_w.predict_proba(X_test_w_scaled)[:, 1]
ensemble_pred_w = (0.6 * et_pred_w + 0.4 * rf_pred_w)

et_log_loss_w = log_loss(y_test_w, et_pred_w)
rf_log_loss_w = log_loss(y_test_w, rf_pred_w)
ensemble_log_loss_w = log_loss(y_test_w, ensemble_pred_w)
print(f"Women's ET Log Loss: {et_log_loss_w:.4f}")
print(f"Women's RF Log Loss: {rf_log_loss_w:.4f}")
print(f"Women's Ensemble Log Loss: {ensemble_log_loss_w:.4f}")

The exact same thing is done for the mens data:

In [ ]:
# Split and Scale for Men
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(X_m, y_m, test_size=0.2, random_state=42)
scaler_m = StandardScaler()
X_train_m_scaled = scaler_m.fit_transform(X_train_m)
X_test_m_scaled = scaler_m.transform(X_test_m)


et_m = ExtraTreesClassifier(
    n_estimators=500,
    max_depth=8,
    min_samples_split=20,
    min_samples_leaf=10,
    max_features='sqrt',
    bootstrap=True,
    random_state=42,
    n_jobs=-1,
    criterion='entropy',
    class_weight={0: 1, 1: 1.2}
)

rf_m = RandomForestClassifier(
    n_estimators=500,
    max_depth=8,
    min_samples_split=20,
    min_samples_leaf=10,
    max_features='sqrt',
    bootstrap=True,
    random_state=42,
    n_jobs=-1,
    criterion='entropy',
    class_weight={0: 1, 1: 1.2}
)


# Fit models for Men
et_m.fit(X_train_m_scaled, y_train_m)
rf_m.fit(X_train_m_scaled, y_train_m)

# Evaluate on the test set for Men
et_pred_m = et_m.predict_proba(X_test_m_scaled)[:, 1]
rf_pred_m = rf_m.predict_proba(X_test_m_scaled)[:, 1]
ensemble_pred_m = (0.6 * et_pred_m + 0.4 * rf_pred_m)

et_log_loss_m = log_loss(y_test_m, et_pred_m)
rf_log_loss_m = log_loss(y_test_m, rf_pred_m)
ensemble_log_loss_m = log_loss(y_test_m, ensemble_pred_m)
print(f"Men's ET Log Loss: {et_log_loss_m:.4f}")
print(f"Men's RF Log Loss: {rf_log_loss_m:.4f}")
print(f"Men's Ensemble Log Loss: {ensemble_log_loss_m:.4f}")


Now this takes every possible matchup in 2025 between the women's and provides a probability (0 to 1) of the first team ID winning that game. In other words, the model learns from all the other seasons, tests what it learns on a portion of those prior seasons, then finally it uses that methodology to make predictions for the possible matchups in 2025. 

In [ ]:
# 2025 Stage 2 Predictions for Women
matchups_2025_w = womens_submission[['Season', 'Team1', 'Team2']]
team_stats_latest_w = team_stats_w.sort_values("Season").drop_duplicates("TeamID", keep="last")
for feature in ["WinPercent", "MOV", "SeedNum", "OffenseStrength", "DefenseStrength", "LastFiveWinRate", 'TurnoverRatio', 'ReboundMargin']:
    matchups_2025_w[f"{feature}_Team1"] = matchups_2025_w['Team1'].map(
        team_stats_latest_w.set_index('TeamID')[feature].fillna(0))
    matchups_2025_w[f"{feature}_Team2"] = matchups_2025_w['Team2'].map(
        team_stats_latest_w.set_index('TeamID')[feature].fillna(0))

matchups_2025_w['SeedDiff'] = matchups_2025_w['SeedNum_Team1'] - matchups_2025_w['SeedNum_Team2']

X_2025_w = matchups_2025_w[[ 
    'WinPercent_Team1', 'WinPercent_Team2', 'MOV_Team1', 'MOV_Team2',
    'SeedNum_Team1', 'SeedNum_Team2', 'OffenseStrength_Team1', 'OffenseStrength_Team2',
    'DefenseStrength_Team1', 'DefenseStrength_Team2', 'LastFiveWinRate_Team1', 
    'LastFiveWinRate_Team2', 'SeedDiff', 'TurnoverRatio_Team1', 'TurnoverRatio_Team2', 'ReboundMargin_Team1',
    'ReboundMargin_Team2'
]].fillna(0)

X_2025_w_scaled = scaler_w.transform(X_2025_w)
et_pred_2025_w = et_w.predict_proba(X_2025_w_scaled)[:, 1]
rf_pred_2025_w = rf_w.predict_proba(X_2025_w_scaled)[:, 1]
ensemble_pred_2025_w = (0.6 * et_pred_2025_w + 0.4 * rf_pred_2025_w)

The same thing for the men:

In [ ]:
matchups_2025_m = mens_submission[['Season', 'Team1', 'Team2']]
team_stats_latest_m = team_stats_m.sort_values("Season").drop_duplicates("TeamID", keep="last")
for feature in ["WinPercent", "MOV", "SeedNum", "OffenseStrength", "DefenseStrength", "LastFiveWinRate", 'TurnoverRatio', 'ReboundMargin']:
    matchups_2025_m[f"{feature}_Team1"] = matchups_2025_m['Team1'].map(
        team_stats_latest_m.set_index('TeamID')[feature].fillna(0))
    matchups_2025_m[f"{feature}_Team2"] = matchups_2025_m['Team2'].map(
        team_stats_latest_m.set_index('TeamID')[feature].fillna(0))

matchups_2025_m['SeedDiff'] = matchups_2025_m['SeedNum_Team1'] - matchups_2025_m['SeedNum_Team2']

X_2025_m = matchups_2025_m[[ 
    'WinPercent_Team1', 'WinPercent_Team2', 'MOV_Team1', 'MOV_Team2',
    'SeedNum_Team1', 'SeedNum_Team2', 'OffenseStrength_Team1', 'OffenseStrength_Team2',
    'DefenseStrength_Team1', 'DefenseStrength_Team2', 'LastFiveWinRate_Team1', 
    'LastFiveWinRate_Team2', 'SeedDiff', 'TurnoverRatio_Team1', 'TurnoverRatio_Team2', 'ReboundMargin_Team1',
    'ReboundMargin_Team2'
]].fillna(0)

X_2025_m_scaled = scaler_m.transform(X_2025_m)
et_pred_2025_m = et_m.predict_proba(X_2025_m_scaled)[:, 1]
rf_pred_2025_m = rf_m.predict_proba(X_2025_m_scaled)[:, 1]
ensemble_pred_2025_m = (0.6 * et_pred_2025_m + 0.4 * rf_pred_2025_m)

Finally the last step is taking all the predictions and merging them into a single csv file (the format required for the competition).

In [ ]:
submission = pd.DataFrame({'ID': sample_submission['ID'], 'Pred': 0.5})
womens_mask = (sample_submission['Team1'].between(3101, 3613)) & (sample_submission['Team2'].between(3101, 3613))
mens_mask = (sample_submission['Team1'].between(1101, 1484)) & (sample_submission['Team2'].between(1101, 1484))
submission.loc[womens_mask, 'Pred'] = ensemble_pred_2025_w
submission.loc[mens_mask, 'Pred'] = ensemble_pred_2025_m

# Save the Submission 
submission.to_csv(r'C:\Users\Gerry\Documents\SUBMISSION1Stage2_Updated.csv', index=False)
print('Submission saved!')